# Run Anaconda as Administrator

In [1]:
#--------------------------------------------------
#Author: Yuehao Wang
#Version: 1.0
#Title: Email script with MD5 Checksum File Exchange
#NOTE: you will need to prepare this file for import
#-------------------------------------------------

#import libraries
import email
import smtplib
import ssl
import xlrd
import xlwt
import s3fs
import hashlib #check sum
import shutil#zip
import time#timestamps
import boto3#s3 access
import os #used to make a directory for save our files
import re #regex library
import mysql.connector

from mysql.connector import errorcode
from shutil import make_archive
from datetime import datetime

In [2]:
#Step 1. Generate Dimension Files. Steps below. In this example, we will generate just one file. 

'''identify location of the dimension files. This is up to you, but this is the location 
where your OUTFILE function in MySQL will post the files'''
output_file_location = 'C:/ProgramData/MySQL/MySQL Server 8.0/Uploads'

# connect to server
connection = mysql.connector.connect(user='root', password='123456',host='localhost', port ='3306')
print('Connected to database.')
cursor = connection.cursor()

#run the sql file 
sql_file = "./dimension_output_example.sql"

with open(sql_file) as f:
    cursor.execute(f.read(), multi=True)
    
#print the results    
result = os.listdir(output_file_location)
print(result)

'''now that you have run your file, you will need to search for it in the folder. 
The following code is a series of regular experessions that allows you to search for the latest file 
in a directory based on the timestamp. Then it provides the latest file as the output.'''


#example of the file name date_dim_20200725101425.csv
files_in_directory = os.listdir(output_file_location)
file_regex = re.compile(r'date_dim_\d{14}.csv$')#regex search for date_dimension file with 14 digits
filtered_files = [ x for x in files_in_directory if file_regex.match(x)]
sorted_files = sorted(filtered_files,reverse=True)
sorted_files[0]

Connected to database.
['DAILY_DIMENSIONS_20210403232819', 'DAILY_DIMENSIONS_20210403233117', 'DAILY_DIMENSIONS_20210403234759', 'date_dim_20210307194630.csv', 'date_dim_20210307195316.csv', 'date_dim_20210307195332.csv', 'date_dim_20210307203403.csv', 'date_dim_20210307225115.csv', 'date_dim_20210308225117.csv', 'date_dim_20210309225117.csv', 'date_dim_20210310225117.csv', 'date_dim_20210311225117.csv', 'date_dim_20210403223217.csv', 'date_dim_20210403224445.csv', 'date_dim_20210403225146.csv', 'date_dim_20210403225217.csv', 'date_dim_20210403225943.csv', 'date_dim_20210403232819.csv', 'date_dim_20210403233117.csv', 'date_dim_20210403234759.csv', 'date_dim_20210403235103.csv', 'fact_orders_20210307203403.csv', 'fact_orders_20210307225115.csv', 'fact_orders_20210308225117.csv', 'fact_orders_20210309225117.csv', 'fact_orders_20210310225117.csv', 'fact_orders_20210311225117.csv', 'orders_dim_20210307225115.csv', 'orders_dim_20210308225117.csv', 'orders_dim_20210309225117.csv', 'orders_di

'date_dim_20210403235103.csv'

In [3]:
#Step 2. Move Files to folder 'checksum'
executiontime = str(re.findall('\d+', sorted_files[0])).replace('[\'','').replace('\']','')
print(executiontime)

directory_name = 'C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/'+'DAILY_DIMENSIONS_%s'%(executiontime)
print(directory_name)

os.mkdir(directory_name) #creates folder structure called 'checksum'
  
os.rename(output_file_location+'/'+sorted_files[0], directory_name+'/'+sorted_files[0])

20210403235103
C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/DAILY_DIMENSIONS_20210403235103


In [4]:
#Step 3. Create check sum procedure
#create md5 checksum file .txt
def file_as_bytes(file):
    with file:
        return file.read()

md5filecontent = hashlib.md5(file_as_bytes(open(directory_name+'/'+sorted_files[0], 'rb'))).hexdigest()
f_name = directory_name+'/'+'md5_checksum.txt'

f = open(f_name, "a")
f.write(md5filecontent)
f.close()

print (os.listdir(directory_name))

['date_dim_20210403235103.csv', 'md5_checksum.txt']


In [9]:
#Step 4: Zip files with checksum in directory called checksum. 
zipName = directory_name
print(zipName)
make_archive(zipName, 'zip', root_dir=directory_name)

file_regex_zip = re.compile(r'DAILY_DIMENSIONS_\d{14}.zip$')
files_in_directory_zip = os.listdir(r'C:\ProgramData\MySQL\MySQL Server 8.0\Uploads')
filtered_files_zip = [ x for x in files_in_directory_zip if file_regex_zip.match(x)]
sorted_files_zip = sorted(filtered_files_zip,reverse=True)
zipFileName = sorted_files_zip[0]


print(zipFileName)

C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/DAILY_DIMENSIONS_20210403235103
DAILY_DIMENSIONS_20210403235103.zip


In [10]:
#Step 5: Load zipped package into AWS S3 with checksum.
s3pathName = 'yw-m10-exercise-update-bucket'  #specify name of your s3 bucket
zipFileNameFullPath = directory_name+'.zip'
print(zipFileNameFullPath)

#print inputs for s3
print(zipFileNameFullPath)
print(s3pathName)
print(zipFileName)

#connect and load the file to s3
s3Resource = boto3.resource('s3')
s3Resource.meta.client.upload_file(zipFileNameFullPath, s3pathName, zipFileName)

#print success message
print("Successfull uploaded file to location: "+str('s3/%s/%s'%(s3pathName,zipFileName)))

C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/DAILY_DIMENSIONS_20210403235103.zip
C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/DAILY_DIMENSIONS_20210403235103.zip
yw-m10-exercise-update-bucket
DAILY_DIMENSIONS_20210403235103.zip
Successfull uploaded file to location: s3/yw-m10-exercise-update-bucket/DAILY_DIMENSIONS_20210403235103.zip
